In [1]:
import sys
import nltk
from nltk.stem.porter import *
from sklearn.feature_extraction import stop_words
import xml.etree.cElementTree as ET
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer,ENGLISH_STOP_WORDS
import zipfile
import os

In [66]:
import pandas as pd
import numpy as np

In [21]:
!ls

Book1.csv          aclImdb            park-nav
NLMCXR_png         ecgen-radiology    talking-data
NLMCXR_reports.tgz ml-latest-small    xml-parser.ipynb


In [36]:
def load_corpus(dir):
    """
    Given a directory containing text files,
    read the text from each file into a dictionary
    with keys as the filename
    """
    fnames = os.listdir(dir)
    txt_dict = {}
    for f in fnames:
        fh = open(os.path.abspath(dir+f), 'r')
        txt = fh.read()
        fh.close()
        txt_dict[f] = txt
    return txt_dict

In [37]:
fpath = 'ecgen-radiology/'

In [38]:
corpus = load_corpus(fpath)

#### Parsing xml

In [41]:
fnames = corpus.keys()

In [49]:
report_dict = {}
for k in fnames: 
    root = ET.fromstring(corpus[k])
    tree = ET.ElementTree(root)
    orientation = []
    for elem in tree.iter(tag='caption'):
        orientation.append(elem.text)
    notes = []
    for elem in tree.iter(tag='AbstractText'):
        notes.append((elem.attrib,elem.text))
    image_id = []
    for elem in tree.iter(tag='parentImage'):
        image_id.append(elem.attrib)
    report_dict[k] = [orientation,notes,image_id]

In [112]:
report_dict["2.xml"][2]

[{'id': 'CXR2_IM-0652-1001'}, {'id': 'CXR2_IM-0652-2001'}]

In [73]:
df = pd.DataFrame.from_dict(list(report_dict.keys()))

In [77]:
df.columns = ["report_name"]

In [84]:
impression = []
for k in fnames:
    impression.append(report_dict[k][1][3][1])

df['impression'] = np.array(impression)

In [94]:
indication = []
for k in fnames:
    indication.append(report_dict[k][1][1][1])

df['indication'] = np.array(indication)

In [97]:
comparison = []
for k in fnames:
    comparison.append(report_dict[k][1][0][1])

df['comparison'] = np.array(comparison)

In [99]:
findings = []
for k in fnames:
    findings.append(report_dict[k][1][2][1])

df['findings'] = np.array(findings)

In [101]:
df.head(2)

,report_name,impression,indication,comparison,findings
0,1.xml,Normal chest x-XXXX.,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...
1,10.xml,No acute cardiopulmonary process.,"XXXX-year-old male, chest pain.",Chest radiographs XXXX.,The cardiomediastinal silhouette is within nor...


In [122]:
report = []
image_id = []
orientation = []
for k in fnames:
    l1 = len(report_dict[k][2])
    l2 = len(report_dict[k][0])
    for l in range(l1):
        report.append(k)
        image_id.append(report_dict[k][2][l]['id'])
        orientation.append(report_dict[k][0][l])

    

In [125]:
df_map = pd.DataFrame([report,image_id,orientation]).T

In [127]:
df_map.columns = ['report_name','id','orientation']

In [128]:
label = pd.merge(df_map,df,'left',on = 'report_name',copy=False)

In [139]:
label.to_csv('openi_labels.csv',index=False)